# lab2 - Set Covering with Evolutionary Algorithms

In [ ]:
import logging
from collections import namedtuple
import random
from matplotlib import pyplot as plt

from Gene import create_dict_genes

In [ ]:
def problem(N, seed=None):
  random.seed(seed)
  return [
    list(set(random.randint(0, N-1) for n in range(random.randint(N//5, N//2))))
    for n in range(random.randint(N, N*5))
  ]

In [ ]:
# Dictionaries to map genes to ids and vice versa
N = 10
id_to_genes = create_dict_genes(problem(N, 42))